---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np

def blight_model():

    import re
    from sklearn import preprocessing
    from adspy_shared_utilities import plot_feature_importances
    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from adspy_shared_utilities import plot_class_regions_for_classifier_subplot
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score  


    data_fields = '''ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
    state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations'''

    disposition_replace = {'Responsible by Default': 'By default',
                       'Responsible by Determination': 'By determination', 
                       'Responsible (Fine Waived) by Deter': 'Fine Waived',
                       'Responsible by Admission': 'By admission',
                       'SET-ASIDE (PENDING JUDGMENT)': 'Pending',
                       'PENDING JUDGMENT': 'Pending',
                       'Not responsible by Dismissal': 'Not responsible',
                       'Not responsible by City Dismissal': 'Not responsible',
                       'Not responsible by Determination': 'Not responsible'
                      } 

    re_pattern1 = '(.*?(?=\s-\s|(?=\s,\s)))'    # .*? match todo hasta ? igual a \s-\s o igual a \s,\s
    re_pattern2 = '[^,\s]+' # tomar y sacar (realizar la división) cuando aparezca un ,\s. El + es indicativo que el resto si va
    data_fields = re.findall(re_pattern1,data_fields) #aplicar pattern1
    data_fields = (list(re.findall(re_pattern2,i)) for i in data_fields) #aplicar pattern2 en lista y formar lista de listas
    data_fields = [x for x in data_fields if x] #eliminar los elementos vacíos ''
    data_fields = [item for data_fields in data_fields for item in data_fields] #nueva lista con cada elemento de la lista de listas 

    df_train = pd.read_csv('train.csv', encoding='iso-8859-1')
    df_train_copy = df_train[pd.notnull(df_train['compliance'])].copy()
    df_train_copy['disposition'].replace(disposition_replace, inplace=True)

    y = df_train_copy['compliance'].tolist()

    df_train_X = df_train_copy[data_fields].copy().set_index('ticket_id')


    ## CONTEO DE CUANTOS VALORES DISTINTOS POR FEATURE Y SELECCIÓN DE FEATURES CON MENORES VALORES
    lst_features_count=[]
    unique_counts = pd.DataFrame.from_records([(col, df_train_X[col].nunique()) for col in df_train_X.columns],
                              columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])
    for feature, value in unique_counts.values:
        if value < 300:
            lst_features_count.append(feature)

    ## ENCODING PARA DECISION TREE Y ENCONTRAR FEATURES MÁS RELEVANTES
    le = preprocessing.LabelEncoder()

    df_train_X_features = pd.DataFrame()                        
    for item in df_train_X[lst_features_count].columns:
        if df_train_X[item].dtypes == np.object:
            le_transform = le.fit_transform(df_train_X[item].astype(str))
            df_train_X_features[item] = le_transform
        else:
            df_train_X_features[item] = df_train_X[item]
    df_train_X_features.fillna(0, inplace=True)

    ## DECISION TREE PARA ENCONTRAR FEATURES MÁS RELEVANTES
    X_train_feature, X_test_feature, y_train_feature, y_test_feature = train_test_split(df_train_X_features.values, y
                                                                                        , random_state = 0)
    clf_1 = DecisionTreeClassifier().fit(X_train_feature, y_train_feature)

    importances = clf_1.feature_importances_
    df_importances = pd.DataFrame([df_train_X_features.columns, importances], 
                                  index=['feature','importance']).T.sort_values(by='importance', ascending=False)
    df_importances_selected = pd.DataFrame()
    for feature, value in df_importances.head(10).values:
        if re.search('^((?!date).)*$',feature):
            df_importances_selected[feature]=feature, value
    features_selected = df_importances_selected.T.index.values
    features_selected = np.array([features_selected[i] for i in [0,1,3,9]], #2 no va
                            dtype='object')   

    df_train_X_model = df_train_X_features[features_selected].sort_index().reset_index(drop=True)
    X_train, X_test, y_train, y_test = train_test_split(df_train_X_model.values, y, random_state = 0)

    clf_test_values = GradientBoostingClassifier().fit(X_train, y_train)

    ## EN CASO DE QUERER TESTEAR PARÁMETROS PARA EL GRADIENT BOOSTING CLASSIFIER
    # grid_test_values = {'max_depth' : [3,5,8]}

    # grid_clf_auc = GridSearchCV(clf_test_values, param_grid = grid_test_values, scoring = 'roc_auc', cv=3, n_jobs = 2)
    # grid_clf_auc.fit(X_train, y_train)
    # grid_clf_auc.grid_scores_, grid_clf_auc.best_params_, grid_clf_auc.best_score_
    ## iter 1 {'learning_rate': 0.2, 'n_estimators': 200}, 0.80940707230458986)
    ## iter 2 for {'learning_rate': 0.2, 'n_estimators': 200} -> {'max_depth': 5}        

    X_test_csv = pd.read_csv('test.csv', encoding='iso-8859-1').set_index('ticket_id')
    X_test_csv = X_test_csv[features_selected]
    X_test_csv['disposition'].replace(disposition_replace, inplace=True)
    le_test = preprocessing.LabelEncoder()
    for item in X_test_csv.columns:
        if X_test_csv[item].dtypes == np.object:
            le_transform = le.fit_transform(X_test_csv[item].astype(str))
            X_test_csv[item] = le_transform        

    clf = GradientBoostingClassifier()

    grid_params = {}
    grid_params = {'n_estimators': [200], 'learning_rate': [0.2], 'max_depth': [5]}

    grid_search = GridSearchCV(clf, param_grid = grid_params, scoring = 'roc_auc', verbose=2)
    grid_search.fit(X_train, y_train)

    preds = grid_search.predict_proba(X_test_csv.values)

    series_blight_prob = pd.Series(data = np.float32([i[1] for i in preds]), index = X_test_csv.index, name='compliance')

        
    return series_blight_prob